In [ ]:
import scanpy as sc

import matplotlib.pyplot as plt
import sys; sys.path.append("../resources/")
from visium_utils import *
from pl_cooccurrence import *

from skimage import io
from MILWRM.MxIF import *

In [ ]:
# make output directories
import os
if not os.path.exists("ST_out"):
    os.mkdir("ST_out")

if not os.path.exists("ST_out/immune_exclusion/"):
    os.mkdir("ST_out/immune_exclusion/")

In [ ]:
sc.set_figure_params(transparent=True, dpi_save=400)
import seaborn as sns; sns.set_style("white")
sc.settings.figdir = "ST_out/immune_exclusion/"

---

In [ ]:
sample_key = pd.read_csv("../resources/ST/visium_sample_key.csv", index_col=0)

sample_key.ws_adata = sample_key.ws_adata.fillna("")
sample_key.TMA = sample_key.TMA.fillna("F")

In [ ]:
rename_dict = {
    "FIB2_VUMCrefNMF30":"FIB2",
    "FIB3_VUMCrefNMF30":"FIB3",
    "MYE1_VUMCrefNMF30":"MYE1",
    "MYE2_VUMCrefNMF30":"MYE2",
    "MYE3_VUMCrefNMF30":"MYE3",
    "MYE4_VUMCrefNMF30":"MYE4",
    "MYE5_VUMCrefNMF30":"MYE5",
    "TL1_VUMCrefNMF30":"TL1",
    "TL2_VUMCrefNMF30":"TL2",
    "TL3_VUMCrefNMF30":"TL3",
    "Immune Exclusion":"IES",  # rename IES
}

markers_thresh = [
    "IES",
    "DDR1",
    "TGFBI",
    "PAK4",
    "DPEP1",
    "FIB2",
    "FIB3",
    "MYE1",
    "MYE2",
    "MYE3",
    "MYE4",
    "MYE5",
    "TL1",
    "TL2",
    "TL3",
]

markers_thresh_dict = dict(zip(markers_thresh, [80] * len(markers_thresh)))

---

In [ ]:
sample_key.patient_name.unique()

In [ ]:
for pat in ["SG00001","SG00002","SG00003","SG00004"]:#sample_key.patient_name.unique():
    print("\nStarting {}".format(pat))
    sample_key_tmp = sample_key.loc[sample_key.patient_name == pat,:].copy()
    
    # check for vTMA sample first
    if any(sample_key_tmp.TMA=="T"):
        # identify index of TMA sample
        sample_key_tma = sample_key_tmp.loc[sample_key_tmp.TMA=="T",:].index
        assert len(sample_key_tma)==1, "Should only be one TMA per sample..."
        print("Detected vTMA sample: {}".format(sample_key_tma[0]))
        a_ws = sc.read("../data/ST/{}_master.h5ad".format(sample_key_tma[0]))
        a_ws.obs.rename(columns=rename_dict, inplace=True)
        block = sample_key.loc[sample_key_tma[0], "block_name"]
        # perform analyses
        immune_excl_cooccurrence(a_ws, pat, block, markers_thresh_dict, TMA_dist=sample_key.loc[sample_key_tma[0], "cooccurrence_dist"], percentiles=True)
        # remove TMA sample and move on
        sample_key_tmp = sample_key_tmp.loc[sample_key_tmp.TMA=="F",:].copy()
    
    # check for whole-slide sample next
    if any(sample_key_tmp.ws_adata!=""):
        # identify path to WS sample
        ws_samples = sample_key_tmp.loc[sample_key_tmp.ws_adata!="", "ws_adata"].unique()
        for ws_sample in ws_samples:
            sample_key_ws = sample_key_tmp.loc[sample_key_tmp.ws_adata==ws_sample, "ws_adata"]
            print("Detected {} whole-slide sample(s) saved in {}".format(len(sample_key_ws), sample_key_ws[0]))
            a_ws = sc.read(sample_key_ws[0])
            a_ws.obs.rename(columns=rename_dict, inplace=True)
            block = sample_key.loc[sample_key.ws_adata == sample_key_ws[0], "block_name"]
            if len(block) > 0:
                block = block[0]
            # perform analyses
            immune_excl_cooccurrence(a_ws, pat, block, markers_thresh_dict, percentiles=True)
        # remove whole-slide sample and move on
        sample_key_tmp = sample_key_tmp.loc[sample_key_tmp.ws_adata=="",:].copy()
        
    # check for "regular" visium samples
    if len(sample_key_tmp) > 0:
        print("Detected {} 'regular' ST samples(s):".format(len(sample_key_tmp)))
        reg_samples = sample_key_tmp.index
        for reg_sample in reg_samples:
            print(reg_sample)
            a_ws = sc.read("../data/ST/{}_master.h5ad".format(reg_sample))
            a_ws.obs.rename(columns=rename_dict, inplace=True)
            block = sample_key.loc[reg_sample, "block_name"]
            # perform analyses
            immune_excl_cooccurrence(a_ws, pat, block, markers_thresh_dict, percentiles=True)
    else:
        print("No 'regular' ST samples for {}".format(pat))
        
    print("DONE.\n")